In [1]:
%pip install faiss-cpu --no-cache

     ---------------------------------------- 10.8/10.8 MB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
samples = pd.read_excel('./initial_data/posts.xlsx').sample(1000)[['title', 'text']]
samples.head()

,title,text
45116,Утренняя разминка на @yaplakal,Утренняя разминка на @yaplakal
62210,"🇫🇷Пока протестующие громили Францию, Макрон с ...","🇫🇷Пока протестующие громили Францию, Макрон с ..."
93790,🟢купил SGZH по 5.992 RUB\nVyacheslavzz,🟢купил SGZH по 5.992 RUB\nVyacheslavzz
8513,**Такой дым поднимается над Новочеркасской ГРЭС,**Такой дым поднимается над Новочеркасской ГРЭ...
11920,Путин: __будем и дальше строить и обновлять об...,Путин: __будем и дальше строить и обновлять об...


In [98]:
samples = samples[~(samples['title'].isna()) & ~(samples['text'].isna())]

preprocesserClass = textPreprocesser(samples.copy(), ['text'])
preprocesserClass.clean()

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = preprocesserClass.df['text'].str.join(' ')
vectorizer = TfidfVectorizer()
embeddings = vectorizer.fit_transform(corpus)
embeddings.shape

(995, 10604)

In [90]:
pd.DataFrame(embeddings.toarray(),columns=vectorizer.get_feature_names_out().tolist()) 

,aaf,aafv,abandoned,abc,abrd,acer,afc,affiliateus,afks,aflt,...,ярост,яруг,ярч,ясиноват,ясн,ят,ях,яхт,яхтен,яхтинг
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
import faiss
import numpy as np

def l2_dists(embeddings: np.ndarray):
    D = X.shape[1]
    N = X.shape[0]

    index = faiss.IndexFlatL2(D)
    index.add(embeddings)

    return index.search(x=embeddings, k=N)

dists, ids = l2_dists(embeddings.toarray())

In [159]:
def dedup(news_dataset: pd.DataFrame, distances: np.ndarray, thresh: float = 1.6) -> tuple:
    non_duplicates_mask = (distances <= thresh) & (distances != 0)
    
    # where L2 is >= 1.6
    non_duplicates = news_dataset[non_duplicates_mask.sum(axis=1) == 0]

    return news_dataset[non_duplicates_mask.sum(axis=1) == 0], news_dataset.shape[0] - non_duplicates.shape[0]

In [160]:
dedup_df, removed_n = dedup(preprocesserClass.df, dists)

In [163]:
removed_n / preprocesserClass.df.shape[0] * 100

53.46733668341709

In [169]:
dedup_df = pd.merge(dedup_df, samples[['text']], left_index=True, right_index=True)
dedup_df.columns = ['title','text_stemmed','text']
dedup_df.to_excel('./initial_data/posts_dedup.xlsx',index=False)